In [1]:
cd ..

/Users/mig/Desktop/pitt/projects/Group-Project-2140/cord_ir


In [2]:
from search.elastic_index_reader import IndexReader
from tqdm.notebook import trange, tqdm
import math

In [3]:
import xml.etree.ElementTree as ET
queryTree = ET.parse('../data/2020-07-16/eval/topics-rnd5.xml')

In [4]:
queryRoot = queryTree.getroot()
queries = []
for child in queryRoot:
    query = {
        'queryNo': child.attrib['number'],
        'query': child.find('query').text,
        'question': child.find('question').text,
        'narrative': child.find('narrative').text
    }
    queries.append(query)

In [5]:
reader = IndexReader()
for query in queries:
    query['query_tokens'] = [t['token'] for t in reader.tokenize(query['query'])['tokens']]
    query['question_tokens'] = [t['token'] for t in reader.tokenize(query['question'])['tokens']]
    query['narrative_tokens'] = [t['token'] for t in reader.tokenize(query['narrative'])['tokens']]

/opt/homebrew/Caskroom/miniforge/base/envs/info-retri/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [6]:
def getRetrievalResults(queries, field):
    results = {}
    for query in tqdm(queries):
        res = reader.search("cord_test", query[field], size=200, fields=[], highlight=False)
        results[query['queryNo']] = res['hits']['hits']
    return results

In [7]:
queryFields = ['query', 'question', 'narrative']
results = {f: getRetrievalResults(queries, f) for f in queryFields}

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
# load the judgments
judgments = {q['queryNo']: [] for q in queries}
with open('../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt', 'r') as qrels:
    for line in qrels:
        [topicId, iteration, cordId, judgment] = line.strip('\n').split(' ')
        judgments[topicId].append({
            'iteration': iteration,
            'cordId': cordId,
            'judgment': judgment
        })

In [9]:
for field in queryFields:
    with open('../data/2020-07-16/eval/%s-result.txt'%field, 'w') as res_file:
        fieldResults = results[field]
        for query in tqdm(queries):
            for (index, result) in enumerate(fieldResults[query['queryNo']]):
                line = ' '.join([query['queryNo'], 'Q0', result['_id'], str(index + 1), str(result['_score']), field])
                res_file.write(line + '\n')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
# use trec_eval to get evaluation
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/query-result.txt
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/question-result.txt
!../data/2020-07-16/eval/trec_eval ../data/2020-07-16/eval/qrels-covid_d5_j0.5-5.txt ../data/2020-07-16/eval/narrative-result.txt

runid                 	all	query
num_q                 	all	50
num_ret               	all	10000
num_rel               	all	26664
num_rel_ret           	all	3774
map                   	all	0.0938
gm_map                	all	0.0542
Rprec                 	all	0.1514
bpref                 	all	0.1417
recip_rank            	all	0.8236
iprec_at_recall_0.00  	all	0.8466
iprec_at_recall_0.10  	all	0.4202
iprec_at_recall_0.20  	all	0.1483
iprec_at_recall_0.30  	all	0.0284
iprec_at_recall_0.40  	all	0.0000
iprec_at_recall_0.50  	all	0.0000
iprec_at_recall_0.60  	all	0.0000
iprec_at_recall_0.70  	all	0.0000
iprec_at_recall_0.80  	all	0.0000
iprec_at_recall_0.90  	all	0.0000
iprec_at_recall_1.00  	all	0.0000
P_5                   	all	0.6640
P_10                  	all	0.6180
P_15                  	all	0.6067
P_20                  	all	0.5840
P_30                  	all	0.5640
P_100                 	all	0.4620
P_200                 	all	0.3774
P_500                 	all	0.1510
P_1000                	

In [11]:
# calculate in Python
ndcg_eval_at = [3, 5, 10, 15, 20, 25, 30]
recall_eval_at = [5, 10, 20, 50, 100, 150]
def getEvaluation(results, judgments):
    queryEval = []
    for queryNo in results:
        # doc to relevant score
        relMap = {}
        # number of docs in each relevant score
        relCount = {}
        totalRel = 0
        for j in judgments[queryNo]:
            score = int(j['judgment'])
            relMap[j['cordId']] = score
            relCount[score] = relCount.get(score, 0) + 1
            if score > 0:
                totalRel += 1
        truePositive = 0
        falsePositive = 0
        precisions = []
        reciprocalRank = 0
        cumulativeGain = 0
        discountedCumulativeGain = 0
        idealDCG = 0
        dcgAt = {}
        idcgAt = {}
        ndcgAt = {}
        index = 1
        recallAt = {}
        for score in sorted(relCount.keys(), reverse=True):
            for i in range(relCount[score]):
                idealDCG += (2 ** score - 1) / (math.log2(1 + index))
                if index in ndcg_eval_at:
                    idcgAt[index] = idealDCG
                index += 1
        for (index, doc) in enumerate(results[queryNo]):
            docId = doc['_id']
            cumulativeGain += relMap.get(docId, 0)
            discountedCumulativeGain += (2 ** (relMap.get(docId, 0)) - 1) / (math.log2(1 + (1 + i)))
            if relMap.get(docId, 0) > 0:
                truePositive += 1
                # recall increase
                precisions.append(truePositive / (truePositive + falsePositive))
                if reciprocalRank == 0:
                    reciprocalRank = 1 / (index+1)
            else:
                falsePositive += 1
            if index + 1 in ndcg_eval_at and (index + 1) in idcgAt:
                dcgAt[index + 1] = discountedCumulativeGain
                ndcgAt[index + 1] = dcgAt[index + 1] / idcgAt[index + 1]
            if index + 1 in recall_eval_at:
                recallAt[index + 1] = truePositive / totalRel
        queryEval.append({
            'AveragePrecision': sum(precisions) / totalRel,
            'ReciprocalRank': reciprocalRank,
            'CG': cumulativeGain,
            'DCG': discountedCumulativeGain,
            'IDCG': idealDCG,
            'nDCG': discountedCumulativeGain / idealDCG,
            'nDCGAt': ndcgAt,
            'recallAt': recallAt
        })
    return {
        'MeanAveragePrecisions': sum(map(lambda e: e['AveragePrecision'], queryEval)) / len(queryEval),
        'MeanReciprocalRank': sum(map(lambda e: e['ReciprocalRank'], queryEval)) / len(queryEval),
        'AverageNDCG': sum(map(lambda e: e['nDCG'], queryEval)) / len(queryEval),
        'AverageNDCGAt': {k: sum(map(lambda e: e['nDCGAt'][k], queryEval)) / len(queryEval) for k in ndcg_eval_at},
        'AverageRecallAt': {k: sum(map(lambda e: e['recallAt'][k], queryEval)) / len(queryEval) for k in recall_eval_at}
    }
            

In [12]:
getEvaluation(results['query'], judgments)

{'MeanAveragePrecisions': 0.0938413425805633,
 'MeanReciprocalRank': 0.8136031746031747,
 'AverageNDCG': 0.14186232674604926,
 'AverageNDCGAt': {3: 0.13364055103725428,
  5: 0.1418463555798589,
  10: 0.1680738131354743,
  15: 0.18464160353165723,
  20: 0.19572503874359487,
  25: 0.20166118601071548,
  30: 0.21199265201748066},
 'AverageRecallAt': {5: 0.007571936093659915,
  10: 0.014347370761125028,
  20: 0.025675868223038946,
  50: 0.05593293689518463,
  100: 0.09603243791764829,
  150: 0.12785061256462518}}

In [13]:
getEvaluation(results['question'], judgments)

{'MeanAveragePrecisions': 0.05846350714788746,
 'MeanReciprocalRank': 0.6930461107519932,
 'AverageNDCG': 0.10352932326145607,
 'AverageNDCGAt': {3: 0.09875194087296979,
  5: 0.11636770250055294,
  10: 0.13835908558004478,
  15: 0.1488085799127936,
  20: 0.15100867206756832,
  25: 0.15061045685942806,
  30: 0.15584868010479463},
 'AverageRecallAt': {5: 0.005727560253085045,
  10: 0.011274666322520223,
  20: 0.019611381677786005,
  50: 0.04029976874013658,
  100: 0.06874504517191328,
  150: 0.09178983181898402}}

In [14]:
getEvaluation(results['narrative'], judgments)

{'MeanAveragePrecisions': 0.0529449798520697,
 'MeanReciprocalRank': 0.6434796152314557,
 'AverageNDCG': 0.10658332315533475,
 'AverageNDCGAt': {3: 0.09286698794380803,
  5: 0.09907587042444932,
  10: 0.12023696099287039,
  15: 0.1276837496201119,
  20: 0.1381839021514124,
  25: 0.1423471423387641,
  30: 0.14730348405985721},
 'AverageRecallAt': {5: 0.005375328395812915,
  10: 0.00957656146663195,
  20: 0.01846469816821065,
  50: 0.03979425169564572,
  100: 0.06755924917514011,
  150: 0.08948080605363223}}

In [15]:
# ML Part 
from search.ml_rank import MlRanker
from search.data_loader import DataLoader
ranker = MlRanker("../data/models/ranker.joblib",
                  '../data/models/tfidf.joblib',
                  "../data/models/docMatrix.joblib")
loader = DataLoader('../data/2020-07-16')
loader.load_metadata_mappings(loader.load_metadata())

In [16]:
def getMlRetrievalResults(queries, field):
    results = {}
    for query in tqdm(queries):
        res = reader.search("cord_test", query[field], size=400, fields=[], highlight=False)
        results[query['queryNo']] = ranker.rank(query[field], res['hits']['hits'], loader)[:200]
#         results[query['queryNo']] = ranker.whole_rank(query[field], loader, size=3000)
    return results
queryFields = ['query', 'question', 'narrative']
mlResults = {f: getMlRetrievalResults(queries, f) for f in queryFields}

  0%|          | 0/50 [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/info-retri/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/info-retri/lib/python3.9/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
getEvaluation(mlResults['query'], judgments)

{'MeanAveragePrecisions': 0.09005768959527849,
 'MeanReciprocalRank': 0.8773431372549021,
 'AverageNDCG': 0.14989653195869732,
 'AverageNDCGAt': {3: 0.13991337162398162,
  5: 0.16019083195779918,
  10: 0.1880138820680024,
  15: 0.19467260831962604,
  20: 0.19345587108302367,
  25: 0.20039637103587843,
  30: 0.20275076498976807},
 'AverageRecallAt': {5: 0.00926449270214008,
  10: 0.015998131184328683,
  20: 0.026009531123613473,
  50: 0.0537778238934053,
  100: 0.09362866234599823,
  150: 0.12833507509372352}}

In [18]:
getEvaluation(mlResults['question'], judgments)

{'MeanAveragePrecisions': 0.06761999829016076,
 'MeanReciprocalRank': 0.8252344877344878,
 'AverageNDCG': 0.11532019227800484,
 'AverageNDCGAt': {3: 0.11905154557103742,
  5: 0.12393254487128674,
  10: 0.14123750446905745,
  15: 0.1582603939056153,
  20: 0.17213461310509906,
  25: 0.1736205311284242,
  30: 0.1879348927923065},
 'AverageRecallAt': {5: 0.007036929542146301,
  10: 0.012438348507519768,
  20: 0.022579426777155062,
  50: 0.04748731389222309,
  100: 0.07728923447092567,
  150: 0.1022987869894738}}

In [19]:
getEvaluation(mlResults['narrative'], judgments)

{'MeanAveragePrecisions': 0.06532275195288008,
 'MeanReciprocalRank': 0.7682640692640693,
 'AverageNDCG': 0.12157516727524953,
 'AverageNDCGAt': {3: 0.10564352676043003,
  5: 0.11180616183951819,
  10: 0.12264224455441847,
  15: 0.1346994667771853,
  20: 0.13848747497440722,
  25: 0.14741789446600842,
  30: 0.15580126111441747},
 'AverageRecallAt': {5: 0.0065333607323246785,
  10: 0.011521895095800096,
  20: 0.01938405775205271,
  50: 0.043768419307058284,
  100: 0.07885855371500826,
  150: 0.10512626881803339}}

In [27]:
def get_diff(res1, res2):
    res = {}
    for key in res1.keys():
        if isinstance(res1[key], dict):
            res[key] = {}
            for key2 in res1[key].keys():
                res[key][key2] = str(((res2[key][key2] - res1[key][key2]) / res1[key][key2]) * 100) + '%'
        else:
            res[key] = str(((res2[key] - res1[key]) / res1[key]) * 100) + '%'
    return res

In [28]:
get_diff(getEvaluation(results['query'], judgments), getEvaluation(mlResults['query'], judgments))

{'MeanAveragePrecisions': '-4.031968087025741%',
 'MeanReciprocalRank': '7.834281458257074%',
 'AverageNDCG': '5.663381813150615%',
 'AverageNDCGAt': {3: '4.693800300912187%',
  5: '12.932638489687829%',
  10: '11.863876091426338%',
  15: '5.432689380998022%',
  20: '-1.159365033282159%',
  25: '-0.6271980244973071%',
  30: '-4.359531776106331%'},
 'AverageRecallAt': {5: '22.35302289327778%',
  10: '11.505665049630414%',
  20: '1.299519446338064%',
  50: '-3.8530302920046946%',
  100: '-2.503087106578917%',
  150: '0.3789285943807709%'}}

In [29]:
get_diff(getEvaluation(results['question'], judgments), getEvaluation(mlResults['question'], judgments))

{'MeanAveragePrecisions': '15.661891646546838%',
 'MeanReciprocalRank': '19.07353276091009%',
 'AverageNDCG': '11.388917308743297%',
 'AverageNDCGAt': {3: '20.556157700414378%',
  5: '6.500809252204546%',
  10: '2.0803974505508083%',
  15: '6.3516592916623305%',
  20: '13.989885976931182%',
  25: '15.277872963676456%',
  30: '20.58805545605948%'},
 'AverageRecallAt': {5: '22.860855778094844%',
  10: '10.321211747749771%',
  20: '15.134298786969147%',
  50: '17.835201979529163%',
  100: '12.428807454625451%',
  150: '11.448931719598496%'}}

In [30]:
get_diff(getEvaluation(results['narrative'], judgments), getEvaluation(mlResults['narrative'], judgments))

{'MeanAveragePrecisions': '23.378556636331425%',
 'MeanReciprocalRank': '19.392137851597585%',
 'AverageNDCG': '14.0658441453037%',
 'AverageNDCGAt': {3: '13.757890828065648%',
  5: '12.849033130399198%',
  10: '2.0004527240926393%',
  15: '5.494604581982252%',
  20: '0.2196875455595247%',
  25: '3.5622437120491726%',
  30: '5.7688907419916555%'},
 'AverageRecallAt': {5: '21.543471416812544%',
  10: '20.313487632762154%',
  20: '4.979012250657087%',
  50: '9.986788146709692%',
  100: '16.72502977434801%',
  150: '17.484713710584714%'}}